# Intermediate Training Summary
The topics covered in the intermediate machine learning course were:
- [Handling Missing Values](#Handling-Missing-Values)
- [Handling Categorical Values](#Handling-Categorical-Data)
- [Pipelines](#Pipelines)
- [Cross-Validation](#Cross-Validation)
- [XGBoost (Gradient Boosting)](#XGBoost-(Gradient-Boosting))
- [Data Leakage](#Data-Leakage)

<a id='Handling-Missing-Values'></a>
## Handling Missing Values
There are generally three different ways of handling missing data (NaN values)

### 1. Drop columns with missing values
The simplest of the three options is to just drop all columns with missing values. This will is a rather extreme approach and not ideal.

In [1]:
# Dropping columns example
# Get names of columns with missing values
missing_cols = [
    col for col in X_train.columns
    if X_train[col].isnull().any()
]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

### 2. Imputation
Imputation fills in missing values with some number, generally the mean value of the column. Imputation isn't exactly right, but leads to more accurate models.

In [0]:
# Imputation Example
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

### 3. Extended Imputation
Sometimes imputating may produce non-ideal values, or maybe the missing values were specifically unique. In this case, adding an additional column to show which rows were initially empty can help with accuracy.

In [0]:
# Extended Imputation Example
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

<a id='Handling-Categorical-Values'></a>
## Handling Categorical Values

Much like handling missing values, there are three main approaches for handling categorical data (data which takes a set limited number of values)

### 1. Drop Categorical Variables
Much like handling missing values, we can just drop the variables/columns that have categorical data. This is simpler, but not a good choice because we may be throwing out useful data.


In [ ]:
# Drop columns in training and validation data
reduced_X_train = X_train.select_dtypes(exclude=['object'])
reduced_X_valid = X_valid.select_dtypes(exclude=['object'])

### 2. Label Encoding
This approach works with categorical data this is 'ordinal'; variables that have an inherent order to them ```Never (0) < Rarely (1) < Always (3)```. 
For tree based models, label encoding works great for ordinal variables.

In [ ]:
# Use skikit-learn's label-encoder
# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])

### 3. One-Hot Encoding
The last approach is the most verbose, taking each categorical value and making it into it's own column. Opposite of ordinal variables, we refer to these non-ordering variables as 'nominal'. One-hot encoding works great as long as there aren't too many values, generally more than 15 different categoricals is as large as we should go.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# NOTE: This becomes much easier with pipelining

<a id='Pipelines'></a>
## Pipelines
Pipelines are an amazing thing that helps us organize and break up our data science into seperate little tasks (and also take some of the coding work off of us).
Some of the advantages of pipelines include:

1. *Cleaner Code*: Accounting for data at each step of preprocessing can get messy. With a pipeline, you won't need to manually keep track of your training and validation data at each step.
2. *Fewer Bugs*: There are fewer opportunities to misapply a step or forget a preprocessing step.
3. *Easier to Productionize*: It can be surprisingly hard to transition a model from a prototype to something deployable at scale. We won't go into the many related concerns here, but pipelines can help.
4. *More Options for Model Validation: You will see an example in the next tutorial, which covers cross-validation.

Here are the four general steps for setting up a pipeline.

### 1. Define Preprocessing Steps
Instead of doing data preprocessing ourselves by hand, we can use the ```ColumnTransformer``` class to bundle our preprocessing steps together.

- imputes missing values in numerical data, and
- imputes missing values and applies a one-hot encoding to categorical data.


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

### 2. Define the Model
For this example, we will define a ```RandomForestRegressor``` for our model pipeline step but note that any valid model will work.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

### 3. Create and Evaluate the Pipeline
Finally, we can use the Pipeline class to bundle up the previous steps into a nice little modeling pipeline. Two things to note are:

- With the pipeline, we preprocess the training data and fit the model in a single line of code. (In contrast, without a pipeline, we have to do imputation, one-hot encoding, and model training in separate steps. This becomes especially messy if we have to deal with both numerical and categorical variables!)
- With the pipeline, we supply the unprocessed features in X_valid to the predict() command, and the pipeline automatically preprocesses the features before generating predictions. (However, without a pipeline, we have to remember to preprocess the validation data before making predictions.)

In [ ]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ]
)

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

<a id='Cross-Validation'></a>
## Cross-Validation
Cross-validation is a very important step in iterating our model and improving it. In cross-validation, we run our modeling process on different subsets of the data to get multiple measures of model quality.

For example, we could begin by dividing the data into 5 pieces, each 20% of the full dataset. In this case, we say that we have broken the data into 5 "folds".
Then, we run one experiment for each fold:

- In Experiment 1, we use the first fold as a validation (or holdout) set and everything else as training data. This gives us a measure of model quality based on a 20% holdout set.
- In Experiment 2, we hold out data from the second fold (and use everything except the second fold for training the model). The holdout set is then used to get a second estimate of model quality.
- We repeat this process, using every fold once as the holdout set. Putting this together, 100% of the data is used as holdout at some point, and we end up with a measure of model quality that is based on all of the rows in the dataset (even if we don't use all rows simultaneously).

Cross-validation helps us reach a more accurate model, but can be very time intensive, taking much time to run multiple models. So when is cross-validation good?

- For small datasets, where extra computational burden isn't a big deal, you should run cross-validation.
- For larger datasets, a single validation set is sufficient. Your code will run faster, and you may have enough data that there's little need to re-use some of it for holdout.

In the example below, we'll use a RandomForestRegressor in a pipeline.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(
    steps=[
        ('preprocessor', SimpleImputer()),
        ('model', RandomForestRegressor(n_estimators=50, random_state=0))
    ]
)

# Use cross_val_score from skikit-learn to make cross validation easy!
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(
    my_pipeline, X, y,
    cv=5,
    scoring='neg_mean_absolute_error'
)

One final note negative MAE values. Scikit-learn has a convention where all metrics are defined so a high number is better. Using negatives here allows them to be consistent with that convention, though negative MAE is almost unheard of elsewhere.


<a id='XGBoost-(Gradient-Boosting)'></a>
## XGBoost (Gradient Boosting)
After learning how to make our work easier with pipelines, and how to make them better with cross-validation, we can now optimize them with gradianet boosting! Previously we have used the RandomForestRegressor model, which is what we call an 'ensemble method' which combines predictions of several models. 

Gradient boosting is a method that goes through cycles to iteratively add models into an ensemble.

It begins by initializing the ensemble with a single model, whose predictions can be pretty naive. (Even if its predictions are wildly inaccurate, subsequent additions to the ensemble will address those errors.)

Then, we start the cycle:
- First, we use the current ensemble to generate predictions for each observation in the dataset. To make a prediction, we add the predictions from all models in the ensemble.
- These predictions are used to calculate a loss function (like mean squared error, for instance).
- Then, we use the loss function to fit a new model that will be added to the ensemble. Specifically, we determine model parameters so that adding this new model to the ensemble will reduce the loss. (Side note: The "gradient" in "gradient boosting" refers to the fact that we'll use gradient descent on the loss function to determine the parameters in this new model.)
- Finally, we add the new model to ensemble, and ...
- ... repeat!

In the below example, we'll use XGBoost. XGBoost stands for extreme gradient boosting, which is an implementation of gradient boosting with several additional features focused on performance and speed. (Scikit-learn has another version of gradient boosting, but XGBoost has some technical advantages.)

Below we import the scikit-learn API for XGBoost (xgboost.XGBRegressor). This allows us to build and fit a model just as we would in scikit-learn.

In [ ]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

A big part of XGBoost is parameter tuning. XGBoost has a few parameters that can dramatically affect accuracy and training speed. The first few we should focus on are:

### n_estimators
```n_estimators``` specifies how many times to go through the modeling cycle described above. It is equal to the number of models that we include in the ensemble.
- Too low a value causes underfitting, which leads to inaccurate predictions on both training data and test data.
- Too high a value causes overfitting, which causes accurate predictions on training data, but inaccurate predictions on test data (which is what we care about).

Typical values range from 100-1000, though this depends a lot on the learning_rate parameter discussed below.

Here is the code to set the number of models in the ensemble:
```
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)
```

### early_stopping_rounds
```early_stopping_rounds``` offers a way to automatically find the ideal value for n_estimators. Early stopping causes the model to stop iterating when the validation score stops improving, even if we aren't at the hard stop for n_estimators. It's smart to set a high value for n_estimators and then use early_stopping_rounds to find the optimal time to stop iterating.

Since random chance sometimes causes a single round where validation scores don't improve, we need to specify a number for how many rounds of straight deterioration to allow before stopping. Setting early_stopping_rounds=5 is a reasonable choice. In this case, we stop after 5 straight rounds of deteriorating validation scores.

When using early_stopping_rounds, we also need to set aside some data for calculating the validation scores - this is done by setting the eval_set parameter.

We can modify the example above to include early stopping:

```
my_model = XGBRegressor(n_estimators=500)
my_model.fit(
    X_train, y_train, 
    early_stopping_rounds=5, 
    eval_set=[(X_valid, y_valid)],
    verbose=False
)
```
If you later want to fit a model with all of your data, set n_estimators to whatever value you found to be optimal when run with early stopping.

### learning_rate
Instead of getting predictions by simply adding up the predictions from each component model, we can multiply the predictions from each model by a small number (known as the learning rate) before adding them in.

This means each tree we add to the ensemble helps us less. So, we can set a higher value for n_estimators without overfitting. If we use early stopping, the appropriate number of trees will be determined automatically.

In general, a small learning rate and large number of estimators will yield more accurate XGBoost models, though it will also take the model longer to train since it does more iterations through the cycle. As default, XGBoost sets learning_rate=0.1.

Modifying the example above to change the learning rate yields the following code:
```
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(
    X_train, y_train, 
    early_stopping_rounds=5, 
    eval_set=[(X_valid, y_valid)], 
    verbose=False
)
```

### n_jobs
On larger datasets where runtime is a consideration, we can use parallelism to build your models faster. It's common to set the parameter n_jobs equal to the number of cores on your machine. On smaller datasets, this won't help.

The resulting model won't be any better, so micro-optimizing for fitting time is typically nothing but a distraction. But, it's useful in large datasets where you would otherwise spend a long time waiting during the fit command.

Here's the modified example:
```
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(
    X_train, y_train, 
    early_stopping_rounds=5, 
    eval_set=[(X_valid, y_valid)], 
    verbose=False
)
```

<a id='Data-Leakage'></a>
## Data Leakage
Lastly, we learn about how to handle data leakage. Data leakage (or leakage) happens when our training data contains information about the target, but similar data will not be available when the model is used for prediction. This leads to high performance on the training set (and possibly even the validation data), but the model will perform poorly in production.

In other words, leakage causes a model to look accurate until we start making decisions with the model, and then the model becomes very inaccurate.

There are two main types of leakage: ```target leakage``` and ```train-test contamination.```

### Target leakage
Target leakage occurs when our predictors include data that will not be available at the time we make predictions. It is important to think about target leakage in terms of the timing or chronological order that data becomes available, not merely whether a feature helps make good predictions.

An example will be helpful. Imagine you want to predict who will get sick with pneumonia. The top few rows of your raw data look like this:


|got_pneumonia|age|weight|male|took_antibiotic_meds|...|
|---|---|---|---|:-:|---|
|False|65|100|False|False|...|
|False|72|130|True|False|...|
|True|58|100|False|True|...|
People take antibiotic medicines after getting pneumonia in order to recover. The raw data shows a strong relationship between those columns, but took_antibiotic_medicine is frequently changed after the value for got_pneumonia is determined. This is target leakage.

The model would see that anyone who has a value of False for took_antibiotic_medicine didn't have pneumonia. Since validation data comes from the same source as training data, the pattern will repeat itself in validation, and the model will have great validation (or cross-validation) scores.

But the model will be very inaccurate when subsequently deployed in the real world, because even patients who will get pneumonia won't have received antibiotics yet when we need to make predictions about their future health.

To prevent this type of data leakage, any variable updated (or created) after the target value is realized should be excluded.

### Train-Test Contamination
A different type of leak occurs when we aren't careful to distinguish training data from validation data.

Recall that validation is meant to be a measure of how the model does on data that it hasn't considered before. We can corrupt this process in subtle ways if the validation data affects the preprocessing behavior. This is sometimes called train-test contamination.

For example, imagine we run preprocessing (like fitting an imputer for missing values) before calling train_test_split(). The end result? Our model may get good validation scores, giving us great confidence in it, but perform poorly when we deploy it to make decisions.

After all, we incorporated data from the validation or test data into how we make predictions, so the may do well on that particular data even if it can't generalize to new data. This problem becomes even more subtle (and more dangerous) when we do more complex feature engineering.

If our validation is based on a simple train-test split, exclude the validation data from any type of fitting, including the fitting of preprocessing steps. This is easier if we use scikit-learn pipelines. When using cross-validation, it's even more critical that we do our preprocessing inside the pipeline!
